In [1]:
from helpers import (
    preprocessing
)
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, IntegerType, DateType, FloatType, ArrayType, LongType, MapType, DateType
import warnings
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
warnings.filterwarnings('ignore')

2022-08-07 11:55:54.142444: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# instancia spark
spark = SparkSession.builder \
    .master("local[*]") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.1.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.memory","24G") \
    .config("spark.driver.memory","24G") \
    .config("spark.executor.cores","8") \
    .config("spark.default.parallelism","8") \
    .config("spark.serializer","org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.execution.arrow.pyspark.enabled","true") \
    .getOrCreate()

22/08/07 11:55:57 WARN Utils: Your hostname, daholive-Dell-G15-5510 resolves to a loopback address: 127.0.1.1; using 192.168.0.114 instead (on interface wlp0s20f3)
22/08/07 11:55:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/daholive/.ivy2/cache
The jars for the packages stored in: /home/daholive/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-019a39aa-9f96-4f5f-9cca-0b1a6e9d3bc2;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/daholive/anaconda3/envs/twiiter_tensor/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 133ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-019a39aa-9f96-4f5f-9cca-0b1a6e9d3bc2
	confs: [default]
	0 artifacts copied, 3 already retrieved (

## Preprocessing Data

In [3]:
# base de dados do twitter ja classificada com sentimentos
path = "/home/daholive/Documents/twitter_ellection_brazil_v2/datasource/raw_kaggle/TweetsWithTheme_v2.csv"

In [4]:
# dataframe twitter com sentimentos classificados
dataframe = spark.read.options(delimiter=';',header='True').csv(path)

In [5]:
# label adjust
dataframe = dataframe.withColumn("sentiment_map", 
    F.when(F.col("sentiment")=="Negativo", 0).otherwise(1)
)

In [6]:
# dataframe features
rdd2 = dataframe.rdd.map(lambda x: (preprocessing(x.tweet_text),len(preprocessing(x.tweet_text).split()),x.sentiment_map))

schema = StructType([       
    StructField('features', StringType(), True),
    StructField('tokens_count', IntegerType(), True),
    StructField('label', IntegerType(), True),
])

df_features = spark.createDataFrame(rdd2, schema = schema)

count_map = F.udf( 
    lambda x: len(x.split()),
    IntegerType()     
)

df_features = df_features \
    .filter(F.col("features")!="-") \
    .filter( count_map(F.col("features"))<30 ) \
    .dropDuplicates(subset = ['features'])

In [7]:
train = df_features.sampleBy("label", fractions={0: 1, 1: 0.87}, seed=10)

In [8]:
# features and labels
features = train.select('features').rdd.flatMap(lambda x: x).collect()
labels = np.array(train.select('label').rdd.flatMap(lambda x: x).collect())

In [9]:
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def tf_idf_features(tokens, num_features):

    try:
        vectorizer = CountVectorizer(max_features=num_features)
        vectorizer.fit(tokens)
        X_vectorizer = vectorizer.transform(tokens)

        tf_idf_transformer = TfidfTransformer()
        tf_idf_transformer.fit(X_vectorizer)
        X_train_tf_idf = tf_idf_transformer.transform(X_vectorizer)

        return csr_matrix(np.array([np.pad(x, (0, num_features-len(x)), 'constant', constant_values=0) for x in X_train_tf_idf.toarray()]))
    
    except:
        return csr_matrix([np.pad(np.array([0]), (0, 1999), 'constant', constant_values=0)])

In [11]:
## MODEL TESTS  LogisticRegression

def train_and_show_scores(x_features: csr_matrix, num_features: int,y: np.array) -> None:

    X_train_tf_idf = tf_idf_features(x_features, num_features)

    X_train, X_valid, y_train, y_valid = train_test_split(
        X_train_tf_idf, y, train_size=0.7, random_state=42
    )

    clf = LogisticRegression(
        class_weight='balanced',
        random_state=42
    )
    
    solvers = ['newton-cg', 'lbfgs', 'liblinear']
    penalty = ['l2']
    c_values = [100, 10, 1.0, 0.1, 0.01]
    max_iter = [10000]
    grid = dict(solver=solvers,penalty=penalty,C=c_values,max_iter=max_iter)
    
    kfold = StratifiedKFold(n_splits = 10, shuffle = True)

    random_search_cv = RandomizedSearchCV(
        estimator=clf,
        param_distributions=grid,
        n_jobs=-1, 
        cv=kfold,
        n_iter=20
    )

    random_search_cv.fit(X_train, y_train)
    y_pred_train = random_search_cv.predict(X_train)
    y_pred_valid = random_search_cv.predict(X_valid)

    print(classification_report_imbalanced(y_train, y_pred_train))
    print(classification_report_imbalanced(y_valid, y_pred_valid))

    return random_search_cv

In [12]:
model_lr = train_and_show_scores(features,2000,labels)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.80      0.74      0.81      0.77      0.78      0.60     19280
          1       0.76      0.81      0.74      0.78      0.78      0.61     19018

avg / total       0.78      0.78      0.78      0.78      0.78      0.60     38298

                   pre       rec       spe        f1       geo       iba       sup

          0       0.76      0.73      0.77      0.74      0.75      0.56      8166
          1       0.74      0.77      0.73      0.76      0.75      0.56      8248

avg / total       0.75      0.75      0.75      0.75      0.75      0.56     16414



## Model prediction with new data

In [14]:
model_lr.predict_proba(tf_idf_features(['Lula é um ótimo presidente'],2000))

array([[0.41394603, 0.58605397]])

In [16]:
model_lr.predict(tf_idf_features(['Lula é um ótimo presidente'],2000))

array([1])

## Model prediction with new tweets data

In [17]:
path_tweets = "/home/daholive/Documents/twitter_ellection_brazil_v2/datasource/trusted/tweets_preprocessing"

In [18]:
df_tweets = spark.read.parquet(path_tweets).toPandas()

In [19]:
df_tweets['tf_idf'] = df_tweets['text_clean'].apply(lambda x: tf_idf_features([x],2000))

In [20]:
df_tweets['sentiment'] = [model_lr.predict(mtx)[0] for mtx in df_tweets['tf_idf']]

In [21]:
df_tweets['sentiment'].value_counts()

1    78495
0    11470
Name: sentiment, dtype: int64

In [23]:
df_tweets.to_pickle('/home/daholive/Documents/twitter_ellection_brazil_v2/datasource/refined/tweets_logistic_regression/tweets_logistic_regression.pkl')